In [ ]:
## NN Dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

## Other Dependencies
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import os

In [ ]:
## Plot style selection - applies style to all matplotlib plots 
plt.style.use(['default','seaborn-whitegrid'])

## Initial Data import and pre-processing for Neural Network model

In [ ]:
## Import dataset csv

churn_df = pd.read_csv('../Resources/clean_churn_db.csv')
attrition_df = pd.read_csv('../Resources/BankChurners.csv', usecols=['Attrition_Flag'])

In [ ]:
print(f"The amount of Attrited Customers/Existing Customers in the dataset is 1628/8500 or {round(1627/8500 * 100,2)}%")

In [ ]:
for column in churn_df.columns:
    print(column)

In [ ]:
churn_df = churn_df.drop(columns=churn_df.columns[0])

In [ ]:
## Define feature values X
X = churn_df.values

## Define target values y
y_df = attrition_df.replace({'Existing Customer':0, 'Attrited Customer':1}).copy()
y = y_df['Attrition_Flag']

In [ ]:
# X = pd.read_csv('../Resources/X.csv')
# y = pd.read_csv('../Resources/y.csv')

In [ ]:
## Split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 7)

In [ ]:
## Instance Scaler
scaler = StandardScaler()

## Fit Scaler
X_scaler = scaler.fit(X_train)

## Scale Data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
len(X_train_scaled[0])

## Initial Modelling Attempt
Using all the availible features in the dataset

In [ ]:
## Layering, beginning with 1 hidden layer
input_features = len(X_train_scaled[0])
hidden_layer_1 = 25
outputs = 1

nn_init = tf.keras.models.Sequential(name='initial')

## First Hidden Layer + Input
nn_init.add(tf.keras.layers.Dense(units = hidden_layer_1, input_dim = input_features, activation = 'relu'))

##Output Layer
nn_init.add(tf.keras.layers.Dense(units = outputs, activation='sigmoid'))

nn_init.summary()

In [ ]:
## Compile and fit
nn_init.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',])
initial_model = nn_init.fit(X_train_scaled, y_train, validation_data = (X_test_scaled, y_test) , epochs=100)

In [ ]:
initial_loss, initial_accuracy = nn_init.evaluate(X_test_scaled, y_test, verbose = 2)

#### Is this accuracy true?
We can verify through validation data. Lets look at the plots for accuracy and loss for both training and validation.

In [ ]:
fig1, initial = plt.subplots(2, figsize=(12, 12))

initial_1 = initial[0].plot(initial_model.history['accuracy'],label='Accuracy', color="navy")
initial_2 = initial[0].plot(initial_model.history['val_accuracy'],label='Validation', color="darkorange")
initial[0].legend(loc='lower right')
initial[0].set_xlim([0,100])
initial[0].set_ylim([.7,1])
initial[0].text(40,.71,f'Evaluated Accuracy: {round(initial_accuracy,4)}',fontsize=12)

initial_3 = initial[1].plot(initial_model.history['loss'],label='Loss', color="navy")
initial_4 = initial[1].plot(initial_model.history['val_loss'],label='Validation', color="darkorange")
initial[1].legend(loc='upper right')
initial[1].set_xlim([0,100])
initial[1].set_ylim([0,.5])
initial[1].text(42,.47,f'Evaluated Loss: {round(initial_loss,4)}',fontsize=12)


plt.savefig("../../static/assets//initial_model.png",facecolor='white')
fig1.tight_layout()

In [ ]:
## Save to CSV for replication as need be.
initial_model_df= pd.DataFrame({
    'Loss': initial_model.history['loss'],
    'Validation Loss': initial_model.history['val_loss'],
    'Accuracy': initial_model.history['accuracy'],
    'Validation Accuracy': initial_model.history['val_accuracy'],
    
})

initial_model_df.to_csv('../../static/assets//initial_model.csv',index=False)

Model seems to be overfitted, adjustments to follow.

## Adjusting Model using L2 regularization

L2 regularization is a standard response to overfitted models. This kind of regularization adds a penalty to the weight values of the nodes on the layer it is activated in.

In [ ]:
## Regularizer import
from tensorflow.keras import regularizers

In [ ]:
## Layering, beginning with 1 hidden layer
input_features = len(X_train_scaled[0])
hidden_layer_1 = 25
outputs = 1

nn_l2 = tf.keras.models.Sequential(name='l2_reg')

## First Hidden Layer + Input. Add regularizer. 
nn_l2.add(tf.keras.layers.Dense(units = hidden_layer_1, input_dim = input_features, activation = 'relu',
                            kernel_regularizer=regularizers.l2(0.001)))

##Output Layer
nn_l2.add(tf.keras.layers.Dense(units = outputs, activation='sigmoid'))

nn_l2.summary()

In [ ]:
nn_l2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',])
l2_reg_model = nn_l2.fit(X_train_scaled, y_train, validation_data = (X_test_scaled, y_test) , epochs=100)

In [ ]:
l2_loss, l2_accuracy = nn_l2.evaluate(X_test_scaled, y_test, verbose = 2)

In [ ]:
## Check for performance/validation. 
fig2, L2 = plt.subplots(2, figsize=(12, 12))

L2[0].plot(l2_reg_model.history['accuracy'], label="Accuracy", color="navy")
L2[0].plot(l2_reg_model.history['val_accuracy'], label="Validation", color="darkorange")
L2[0].legend(loc='lower right')
L2[0].set_xlim([0,100])
L2[0].set_ylim([.7,1])
L2[0].text(40,.71,f'Evaluated Accuracy: {round(l2_accuracy,4)}',fontsize=12)


L2[1].plot(l2_reg_model.history['loss'], label="Loss", color="navy")
L2[1].plot(l2_reg_model.history['val_loss'], label="Validation", color="darkorange")
L2[1].legend(loc='upper right')
L2[1].set_xlim([0,100])
L2[1].set_ylim([0,.5])
L2[1].text(42,.47,f'Evaluated Loss: {round(l2_loss,4)}',fontsize=12)


plt.savefig("../../static/assets//L2_model.png",facecolor='white')
fig2.tight_layout()

In [ ]:
L2_model_df= pd.DataFrame({
    'Loss': l2_reg_model.history['loss'],
    'Validation Loss': l2_reg_model.history['val_loss'],
    'Accuracy': l2_reg_model.history['accuracy'],
    'Validation Accuracy': l2_reg_model.history['val_accuracy'],
    
})

L2_model_df.to_csv('../Outputs//neural_network/L2_model.csv',index=False)

In [ ]:
## Looks good at first, but the model still finds out a pattern early and does not adapt to new information

## Adjusting model using Dropout

Dropout is another standard method in response to overfitting. Drops nodes by probability to decrease any given node from correct the mistakes of other nodes. 

In [ ]:
## Layering, beginning with 1 hidden layer
input_features = len(X_train_scaled[0])
hidden_layer_1 = 25
outputs = 1

nn_dropout = tf.keras.models.Sequential(name='dropout')

## First Hidden Layer + Input

nn_dropout.add(tf.keras.layers.Dense(units = hidden_layer_1, input_dim = input_features, activation = 'relu'))

## Add dropout
nn_dropout.add(tf.keras.layers.Dropout(.2))

##Output Layer
nn_dropout.add(tf.keras.layers.Dense(units = outputs, activation='sigmoid'))


nn_dropout.summary()

In [ ]:
nn_dropout.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',])
dropout_model = nn_dropout.fit(X_train_scaled, y_train, validation_data = (X_test_scaled, y_test) , epochs=100)

In [ ]:
dropout_loss, dropout_accuracy = nn_dropout.evaluate(X_test_scaled, y_test, verbose = 2)

In [ ]:
fig3, dropout = plt.subplots(2, figsize=(12, 12))

dropout[0].plot(dropout_model.history['accuracy'], label='Accuracy', color="navy")
dropout[0].plot(dropout_model.history['val_accuracy'], label='Validation', color="darkorange")
dropout[0].legend(loc='lower right')
dropout[0].set_xlim([0,100])
dropout[0].set_ylim([.7,1])
dropout[0].text(40,.71,f'Evaluated Accuracy: {round(dropout_accuracy,4)}',fontsize=12)

dropout[1].plot(dropout_model.history['loss'], label='Loss', color="navy")
dropout[1].plot(dropout_model.history['val_loss'], label='Validation', color="darkorange")
dropout[1].legend(loc='upper right')
dropout[1].set_xlim([0,100])
dropout[1].set_ylim([0,.5])
dropout[1].text(42,.47,f'Evaluated Loss: {round(dropout_loss,4)}',fontsize=12)

plt.savefig("../../static/assets//dropout_model.png",facecolor='white')
fig3.tight_layout()

In [ ]:
dropout_model_df= pd.DataFrame({
    'Loss': dropout_model.history['loss'],
    'Validation Loss': dropout_model.history['val_loss'],
    'Accuracy': dropout_model.history['accuracy'],
    'Validation Accuracy': dropout_model.history['val_accuracy'],
    
})

dropout_model_df.to_csv('../Outputs//neural_network/dropout_model.csv',index=False)

Looking better as well. The next step will combine both L2 and dropout.

## Using both L2 and Dropout

In [ ]:
## Layering, beginning with 1 hidden layer
input_features = len(X_train_scaled[0])
hidden_layer_1 = 25
outputs = 1

nn_l2_dropout = tf.keras.models.Sequential(name='dropout_and_l2')

## First Hidden Layer + Input + regularizer.
nn_l2_dropout.add(tf.keras.layers.Dense(units = hidden_layer_1, input_dim = input_features, activation = 'relu',
                             kernel_regularizer=regularizers.l2(0.001)))

## Add dropout
nn_l2_dropout.add(tf.keras.layers.Dropout(.2))


##Output Layer
nn_l2_dropout.add(tf.keras.layers.Dense(units = outputs, activation='sigmoid'))


nn_l2_dropout.summary()

In [ ]:
nn_l2_dropout.compile(
    loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 
    tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
dropout_l2_model = nn_l2_dropout.fit(X_train_scaled, y_train, validation_data = (X_test_scaled, y_test) , epochs=100)

In [ ]:
dropl2_loss, dropl2_accuracy, dropl2_prec, dropl2_recall = nn_l2_dropout.evaluate(X_test_scaled, y_test, verbose = 2)

In [ ]:
fig4, L2_drop = plt.subplots(2, figsize=(12, 12))

L2_drop[0].plot(dropout_l2_model.history['accuracy'], label='Accuracy', c='orange')
L2_drop[0].plot(dropout_l2_model.history['val_accuracy'], label='Validation', color="navy")
L2_drop[0].legend()
L2_drop[0].legend(loc='lower right')
L2_drop[0].set_xlim([0,100])
L2_drop[0].set_ylim([.7,1])
L2_drop[0].text(40,.71,f'Evaluated Accuracy: {round(dropl2_accuracy,4)}',fontsize=12)

L2_drop[1].plot(dropout_l2_model.history['loss'], label='Loss', c='orange')
L2_drop[1].plot(dropout_l2_model.history['val_loss'], label='Validation', color="navy")
L2_drop[1].legend()
L2_drop[1].legend(loc='upper right')
L2_drop[1].set_xlim([0,100])
L2_drop[1].set_ylim([0,.5])
L2_drop[1].text(42,.47,f'Evaluated Loss: {round(dropl2_loss,4)}',fontsize=12)


plt.savefig("../../static/assets//dropout_L2_model.png",facecolor='white')
fig4.tight_layout()

In [ ]:
dropout_l2_df= pd.DataFrame({
    'Loss': dropout_l2_model.history['loss'],
    'Validation Loss': dropout_l2_model.history['val_loss'],
    'Accuracy': dropout_l2_model.history['accuracy'],
    'Validation Accuracy': dropout_l2_model.history['val_accuracy'],
    
})

dropout_l2_df.to_csv('../Outputs//neural_network/dropout_l2_history.csv',index=False)
dropout_l2_df.to_hdf('../Outputs//neural_network/dropout_l2_model.h5',key='dl2_model')

Good convergence with validation and training. The problem of overfitting has been resolved.

## Continued Visualization

In order to make the best judgements about the model, additional visualizations should be made.

### Confusion Matrix

In [ ]:
y_pred_nn = nn_l2_dropout.predict(X_test_scaled) > .5

In [ ]:
y_pred_nn.shape

In [ ]:
y_test.shape

In [ ]:
matrix = tf.math.confusion_matrix(y_test,y_pred_nn)

In [ ]:
matrix_df = pd.DataFrame(matrix)
matrix_df = matrix_df.rename(columns={0:"Positive_r",1:"Negative_r"}, index={0:"Positive_c",1:"Negative_c"})
matrix_df.to_csv("../Outputs//neural_network/con_matrix.csv")

### Model Architecture

In [ ]:
tf.keras.utils.plot_model(nn_l2_dropout, to_file='../Outputs//neural_network/dropout_l2_arch.png', show_shapes=True, 
                          show_layer_names=False)

### ROC curve and AOC-ROC analysis.

In [ ]:
## Import methods for ROC curve, AUC and AUC score.
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

## Utilize method from Tensorflow Keras documentation

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(37):
    fpr[i], tpr[i], _ = roc_curve(y_test, y_pred_nn)
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_pred_nn.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
plt.figure()
plt.plot(
    fpr[2],
    tpr[2],
    color="darkorange",
    lw=2,
    label="ROC curve (area = %0.4f)" % roc_auc[2],
)
plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC")
plt.legend(loc="lower right")

plt.savefig("../../static/assets//nn_ROC.png",facecolor='white')
plt.show()

In [ ]:
## Export false positive rate, true positive rate, and coresponding auc value for use as needed 
nn_roc = pd.DataFrame({"fpr":fpr[2],'tpr':tpr[2],'AUC':roc_auc[2]})
nn_roc.to_csv('../Outputs//neural_network/nn_roc.csv',index=False)